In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("theblackmamba31/landscape-image-colorization")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/landscape-image-colorization


In [ ]:
#!/usr/bin/env python3
"""
Photo Colorization System - Google Colab Version
Optimized for /kaggle/input/landscape-image-colorization/landscape Images path
"""

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import time
import json
from datetime import datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

class ColorizationDataset(Dataset):
    """Dataset for training colorization model - Colab optimized"""

    def __init__(self, data_dir, transform=None, dataset_type='auto'):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.dataset_type = dataset_type
        self.gray_files, self.color_files = self._get_image_pairs()

    def _get_image_pairs(self):
        """Get paired gray and color images"""
        # Check if this is a landscape dataset with gray/color folders
        gray_dir = self.data_dir / 'gray'
        color_dir = self.data_dir / 'color'

        if gray_dir.exists() and color_dir.exists():
            print(f"📁 Found landscape dataset structure: gray/ and color/ folders")
            self.dataset_type = 'landscape'

            # Get gray images
            gray_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            gray_files = []
            for ext in gray_extensions:
                gray_files.extend(gray_dir.glob(f'*{ext}'))
                gray_files.extend(gray_dir.glob(f'*{ext.upper()}'))

            # Get corresponding color images
            color_files = []
            valid_pairs = []
            for gray_file in gray_files:
                color_file = color_dir / gray_file.name
                if color_file.exists():
                    valid_pairs.append((gray_file, color_file))
                else:
                    print(f"⚠️ No color counterpart for {gray_file.name}")

            gray_files = [pair[0] for pair in valid_pairs]
            color_files = [pair[1] for pair in valid_pairs]

            print(f"📊 Found {len(gray_files)} valid gray-color pairs")

        else:
            # Fallback to single directory (convert color to gray)
            print(f"📁 Using single directory mode (converting color to gray)")
            self.dataset_type = 'single'

            extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            color_files = []
            for ext in extensions:
                color_files.extend(self.data_dir.glob(f'*{ext}'))
                color_files.extend(self.data_dir.glob(f'*{ext.upper()}'))

            gray_files = color_files  # Will be converted during processing

        return gray_files, color_files

    def __len__(self):
        return len(self.gray_files)

    def __getitem__(self, idx):
        if self.dataset_type == 'landscape':
            # Load gray and color images separately
            gray_path = self.gray_files[idx]
            color_path = self.color_files[idx]

            # Load gray image
            gray_img = cv2.imread(str(gray_path), cv2.IMREAD_GRAYSCALE)
            if gray_img is None:
                raise ValueError(f"Could not load gray image: {gray_path}")

            # Load color image
            color_img = cv2.imread(str(color_path))
            if color_img is None:
                raise ValueError(f"Could not load color image: {color_path}")

            color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)

        else:
            # Load color image and convert to gray
            img_path = self.gray_files[idx]  # This is actually a color image in single mode
            color_img = cv2.imread(str(img_path))

            if color_img is None:
                raise ValueError(f"Could not load image: {img_path}")

            color_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB)

            # Convert to grayscale
            gray_img = cv2.cvtColor(color_img, cv2.COLOR_RGB2GRAY)

        # Resize to 256x256
        color_img = cv2.resize(color_img, (256, 256))
        gray_img = cv2.resize(gray_img, (256, 256))

        # Normalize
        gray_img = gray_img.astype(np.float32) / 255.0
        color_img = color_img.astype(np.float32) / 255.0

        # Convert to tensors
        gray_tensor = torch.from_numpy(gray_img).unsqueeze(0)  # (1, H, W)
        color_tensor = torch.from_numpy(color_img).permute(2, 0, 1)  # (3, H, W)

        return gray_tensor, color_tensor

class ColorizationModel(nn.Module):
    """U-Net based colorization model - Colab optimized"""

    def __init__(self):
        super().__init__()

        # Encoder
        self.enc1 = self._make_layer(1, 64)
        self.enc2 = self._make_layer(64, 128)
        self.enc3 = self._make_layer(128, 256)
        self.enc4 = self._make_layer(256, 512)

        # Decoder
        self.dec4 = self._make_layer(512, 256)
        self.dec3 = self._make_layer(256, 128)
        self.dec2 = self._make_layer(128, 64)
        self.dec1 = nn.Conv2d(64, 3, kernel_size=1)

        # Pooling and upsampling
        self.pool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # Activation
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def _make_layer(self, in_channels, out_channels):
        """Create a convolutional layer"""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Decoder with skip connections
        dec4 = self.dec4(self.upsample(enc4))
        dec3 = self.dec3(self.upsample(dec4))
        dec2 = self.dec2(self.upsample(dec3))
        dec1 = self.dec1(dec2)

        return self.sigmoid(dec1)

class Colorizer:
    """Main colorizer class with Colab optimizations"""

    def __init__(self, device='auto'):
        self.device = self._get_device(device)
        self.model = ColorizationModel().to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.training_history = []
        print(f"🎨 Colorizer initialized on {self.device}")

    def _get_device(self, device):
        """Get best available device for Colab"""
        if device == 'auto':
            if torch.cuda.is_available():
                return 'cuda'
            elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
                return 'mps'
            else:
                return 'cpu'
        return device

    def analyze_dataset(self, data_dir):
        """Analyze the dataset before training - Colab optimized"""
        print("📊 Analyzing dataset...")

        data_path = Path(data_dir)
        if not data_path.exists():
            raise ValueError(f"Dataset directory does not exist: {data_dir}")

        # Check if this is a landscape dataset
        gray_dir = data_path / 'gray'
        color_dir = data_path / 'color'

        if gray_dir.exists() and color_dir.exists():
            # Landscape dataset structure
            print(f"📁 Analyzing landscape dataset structure...")

            # Get gray and color files
            extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            gray_files = []
            color_files = []

            for ext in extensions:
                gray_files.extend(gray_dir.glob(f'*{ext}'))
                gray_files.extend(gray_dir.glob(f'*{ext.upper()}'))
                color_files.extend(color_dir.glob(f'*{ext}'))
                color_files.extend(color_dir.glob(f'*{ext.upper()}'))

            # Find matching pairs
            gray_names = {f.name for f in gray_files}
            color_names = {f.name for f in color_files}
            matching_names = gray_names.intersection(color_names)

            valid_pairs = len(matching_names)

            if valid_pairs == 0:
                raise ValueError(f"No matching gray-color pairs found in {data_dir}")

            print(f" Found {valid_pairs} gray-color pairs")

            # Analyze sample images
            sizes = []
            formats = {}
            corrupt_files = []

            # Sample a few images for analysis
            sample_files = list(matching_names)[:min(10, len(matching_names))]

            for name in sample_files:
                try:
                    gray_path = gray_dir / name
                    color_path = color_dir / name

                    # Check gray image
                    gray_img = cv2.imread(str(gray_path), cv2.IMREAD_GRAYSCALE)
                    if gray_img is None:
                        corrupt_files.append(f"gray/{name}")
                        continue

                    # Check color image
                    color_img = cv2.imread(str(color_path))
                    if color_img is None:
                        corrupt_files.append(f"color/{name}")
                        continue

                    h, w = color_img.shape[:2]
                    sizes.append((w, h))

                    ext = Path(name).suffix.lower()
                    formats[ext] = formats.get(ext, 0) + 1

                except Exception as e:
                    corrupt_files.append(name)
                    print(f"⚠️ Error analyzing {name}: {e}")

            analysis = {
                'dataset_type': 'landscape',
                'total_pairs': valid_pairs,
                'gray_files': len(gray_files),
                'color_files': len(color_files),
                'matching_pairs': valid_pairs,
                'corrupt_files': len(corrupt_files),
                'formats': formats,
                'sample_sizes': sizes
            }

        else:
            # Single directory structure
            print(f"📁 Analyzing single directory structure...")

            extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            image_files = []
            for ext in extensions:
                image_files.extend(data_path.glob(f'*{ext}'))
                image_files.extend(data_path.glob(f'*{ext.upper()}'))

            if not image_files:
                raise ValueError(f"No image files found in {data_dir}")

            print(f"📁 Found {len(image_files)} images")

            # Analyze image properties
            sizes = []
            formats = {}
            corrupt_files = []

            for img_file in image_files:
                try:
                    img = cv2.imread(str(img_file))
                    if img is None:
                        corrupt_files.append(str(img_file))
                        continue

                    h, w = img.shape[:2]
                    sizes.append((w, h))

                    ext = img_file.suffix.lower()
                    formats[ext] = formats.get(ext, 0) + 1

                except Exception as e:
                    corrupt_files.append(str(img_file))
                    print(f"⚠️ Error analyzing {img_file}: {e}")

            valid_images = len(image_files) - len(corrupt_files)

            analysis = {
                'dataset_type': 'single',
                'total_files': len(image_files),
                'valid_images': valid_images,
                'corrupt_files': len(corrupt_files),
                'formats': formats,
                'image_sizes': sizes
            }

        # Print analysis
        print(f"✅ Dataset Analysis Complete:")
        if analysis['dataset_type'] == 'landscape':
            print(f"   📊 Dataset type: Landscape (gray/color pairs)")
            print(f"   📁 Total pairs: {analysis['total_pairs']}")
            print(f"   📋 Formats: {', '.join(f'{k}({v})' for k, v in formats.items())}")
        else:
            print(f"   📊 Dataset type: Single directory")
            print(f"   📁 Total files: {analysis['total_files']}")
            print(f"   ✅ Valid images: {analysis['valid_images']}")
            print(f"   ❌ Corrupt files: {analysis['corrupt_files']}")
            print(f"   📋 Formats: {', '.join(f'{k}({v})' for k, v in formats.items())}")

        if corrupt_files:
            print(f"   ⚠️ Corrupt files found: {len(corrupt_files)}")

        # Save analysis report
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_path = f"dataset_analysis_{timestamp}.json"
        with open(report_path, 'w') as f:
            json.dump(analysis, f, indent=2, default=str)
        print(f"   Analysis saved to: {report_path}")

        return analysis

    def train(self, data_dir, epochs=10, batch_size=8, save_path='model.pth'):
        """Train the colorization model - Colab optimized with progress bars"""
        print(f"🚀 Starting training with {epochs} epochs...")

        # Analyze dataset first
        try:
            analysis = self.analyze_dataset(data_dir)
            # Check for valid images based on dataset type
            if analysis['dataset_type'] == 'landscape':
                if analysis['total_pairs'] < 5:
                    print("⚠️ Warning: Very few image pairs for training. Consider adding more data.")
            else:
                if analysis['valid_images'] < 5:
                    print("⚠️ Warning: Very few images for training. Consider adding more data.")
        except Exception as e:
            print(f"❌ Dataset analysis failed: {e}")
            return None

        # Create dataset and dataloader
        try:
            dataset = ColorizationDataset(data_dir)
            if len(dataset) == 0:
                raise ValueError("Dataset is empty")

            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            print(f"📊 Dataset loaded: {len(dataset)} images, {len(dataloader)} batches")

        except Exception as e:
            print(f"❌ Failed to create dataset: {e}")
            return None

        # Training setup
        start_time = time.time()
        best_loss = float('inf')

        # Training loop with progress bars
        for epoch in range(epochs):
            self.model.train()
            epoch_start = time.time()
            total_loss = 0
            batch_count = 0

            # Progress bar for batches
            pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")

            try:
                for batch_idx, (gray, color) in enumerate(pbar):
                    gray = gray.to(self.device)
                    color = color.to(self.device)

                    # Forward pass
                    self.optimizer.zero_grad()
                    output = self.model(gray)
                    loss = self.criterion(output, color)

                    # Backward pass
                    loss.backward()
                    self.optimizer.step()

                    total_loss += loss.item()
                    batch_count += 1

                    # Update progress bar
                    pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

                # Calculate epoch metrics
                avg_loss = total_loss / batch_count if batch_count > 0 else 0
                epoch_time = time.time() - epoch_start

                # Track training history
                epoch_info = {
                    'epoch': epoch + 1,
                    'avg_loss': avg_loss,
                    'time': epoch_time,
                    'timestamp': datetime.now().isoformat()
                }
                self.training_history.append(epoch_info)

                # Update best loss
                if avg_loss < best_loss:
                    best_loss = avg_loss
                    # Save best model
                    best_model_path = save_path.replace('.pth', '_best.pth')
                    torch.save(self.model.state_dict(), best_model_path)

                print(f"✅ Epoch {epoch+1}/{epochs} completed in {epoch_time:.1f}s. Average Loss: {avg_loss:.4f}")

            except Exception as e:
                print(f"❌ Training error at epoch {epoch+1}: {e}")
                break

        # Training completed
        total_time = time.time() - start_time

        # Save final model
        try:
            torch.save(self.model.state_dict(), save_path)
            print(f"💾 Final model saved to {save_path}")

            # Save training history
            history_path = save_path.replace('.pth', '_history.json')
            training_summary = {
                'total_epochs': len(self.training_history),
                'total_time': total_time,
                'best_loss': best_loss,
                'final_loss': self.training_history[-1]['avg_loss'] if self.training_history else 0,
                'dataset_info': analysis,
                'training_history': self.training_history
            }

            with open(history_path, 'w') as f:
                json.dump(training_summary, f, indent=2)
            print(f"📊 Training history saved to {history_path}")

            # Print training summary
            print(f"\n🎉 Training Complete!")
            print(f"   ⏱️ Total time: {total_time:.1f} seconds")
            print(f"   📊 Epochs completed: {len(self.training_history)}")
            print(f"   🏆 Best loss: {best_loss:.4f}")
            print(f"   📈 Final loss: {self.training_history[-1]['avg_loss']:.4f}")

            return training_summary

        except Exception as e:
            print(f"❌ Failed to save model: {e}")
            return None

    def load_model(self, model_path):
        """Load a trained model"""
        if os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path, map_location=self.device))
            self.model.eval()
            print(f"✅ Model loaded from {model_path}")
        else:
            print(f"⚠️ No model found at {model_path}, using untrained model")

    def colorize(self, input_path, output_path=None):
        """Colorize a single image"""
        print(f" Processing: {input_path}")

        # Load and preprocess image
        image = cv2.imread(input_path)
        if image is None:
            raise ValueError("Could not load image")

        # Convert to grayscale if needed
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image

        # Resize for processing
        original_size = (image.shape[1], image.shape[0])
        gray_resized = cv2.resize(gray, (256, 256))

        # Convert to tensor
        gray_tensor = torch.from_numpy(gray_resized).float().unsqueeze(0).unsqueeze(0)
        gray_tensor = gray_tensor / 255.0
        gray_tensor = gray_tensor.to(self.device)

        # Colorize
        with torch.no_grad():
            self.model.eval()
            output = self.model(gray_tensor)

        # Convert back to image
        colorized = output.squeeze().cpu().numpy()
        colorized = np.transpose(colorized, (1, 2, 0))
        colorized = (colorized * 255).astype(np.uint8)
        colorized = cv2.cvtColor(colorized, cv2.COLOR_RGB2BGR)

        # Resize back to original size
        colorized = cv2.resize(colorized, original_size)

        # Save result
        if output_path:
            cv2.imwrite(output_path, colorized)
            print(f"💾 Saved to: {output_path}")

        return colorized

    def batch_process(self, input_dir, output_dir):
        """Process multiple images - Colab optimized"""
        input_path = Path(input_dir)
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)

        # Check if this is a landscape dataset
        gray_dir = input_path / 'gray'
        color_dir = input_path / 'color'

        if gray_dir.exists() and color_dir.exists():
            # Use gray images for processing (they will be colorized)
            extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            image_files = []
            for ext in extensions:
                image_files.extend(gray_dir.glob(f'*{ext}'))
                image_files.extend(gray_dir.glob(f'*{ext.upper()}'))
        else:
            # Find image files in root directory
            extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}
            image_files = []
            for ext in extensions:
                image_files.extend(input_path.glob(f'*{ext}'))
                image_files.extend(input_path.glob(f'*{ext.upper()}'))

        if not image_files:
            print(f"No image files found in {input_dir}")
            return

        print(f"📁 Found {len(image_files)} images to process")

        results = []
        # Progress bar for batch processing
        for i, img_file in enumerate(tqdm(image_files, desc="Processing images")):
            try:
                output_file = output_path / f"colorized_{img_file.name}"
                colorized = self.colorize(str(img_file), str(output_file))
                results.append((str(img_file), str(output_file), True))
            except Exception as e:
                print(f"❌ Error processing {img_file.name}: {e}")
                results.append((str(img_file), None, False))

        successful = sum(1 for _, _, success in results if success)
        print(f"🎉 Batch processing complete! {successful}/{len(results)} successful")
        return results

    def test(self, test_dir, output_dir='test_results'):
        """Test the model on a directory of images"""
        print(" Running model test...")

        # Create output directory
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)

        # Process test images
        results = self.batch_process(test_dir, output_dir)

        # Generate test report
        if results:
            successful = sum(1 for _, _, success in results if success)
            total = len(results)
            success_rate = (successful / total) * 100

            print(f"\n📊 Test Results:")
            print(f"Total images: {total}")
            print(f"Successful: {successful}")
            print(f"Failed: {total - successful}")
            print(f"Success rate: {success_rate:.1f}%")

        return results

# ========================================
# Colab Usage Functions - Correct Path
# ========================================

def colab_train():
    """Simple function for Colab training"""
    print("🚀 Starting Colab training...")

    # Initialize colorizer
    colorizer = Colorizer(device='auto')

    # Use the correct Kaggle path with space
    dataset_path = "/kaggle/input/landscape-image-colorization/landscape Images"

    if not Path(dataset_path).exists():
        print(f"❌ Dataset not found at {dataset_path}")
        print("Available directories:")
        for item in Path('/kaggle/input/landscape-image-colorization').iterdir():
            print(f"  - {item}")
        return

    print(f"📁 Using dataset: {dataset_path}")

    # Train on landscape dataset
    colorizer.train(dataset_path, epochs=20, batch_size=16, save_path='model.pth')

    print(" Training completed!")

def colab_test():
    """Simple function for Colab testing"""
    print("🧪 Starting Colab testing...")

    # Initialize colorizer
    colorizer = Colorizer(device='auto')

    # Load model
    colorizer.load_model('model.pth')

    # Use the correct Kaggle path with space
    dataset_path = "/kaggle/input/landscape-image-colorization/landscape Images"

    if not Path(dataset_path).exists():
        print(f"❌ Dataset not found at {dataset_path}")
        return

    # Test on gray images
    test_dir = f"{dataset_path}/gray"
    colorizer.test(test_dir, 'test_results')

    print("🎉 Testing completed!")

def colab_colorize_single():
    """Colorize a single image"""
    print("🎨 Starting single image colorization...")

    # Initialize colorizer
    colorizer = Colorizer(device='auto')

    # Load model
    colorizer.load_model('model.pth')

    # Use the correct Kaggle path with space
    dataset_path = "/kaggle/input/landscape-image-colorization/landscape Images"

    if not Path(dataset_path).exists():
        print(f"❌ Dataset not found at {dataset_path}")
        return

    # Find a sample image
    gray_dir = Path(dataset_path) / "gray"
    if gray_dir.exists():
        gray_files = list(gray_dir.glob('*.jpg'))
        if gray_files:
            sample_image = str(gray_files[0])
            print(f"🎨 Colorizing: {sample_image}")

            # Colorize
            output_path = "colorized_sample.jpg"
            colorizer.colorize(sample_image, output_path)

            print(f"🎉 Colorization completed! Saved to: {output_path}")
        else:
            print("❌ No gray images found")
    else:
        print("❌ Gray directory not found")

def colab_analyze_dataset():
    """Analyze the dataset"""
    print("📊 Analyzing dataset...")

    # Initialize colorizer
    colorizer = Colorizer(device='auto')

    # Use the correct Kaggle path with space
    dataset_path = "/kaggle/input/landscape-image-colorization/landscape Images"

    if not Path(dataset_path).exists():
        print(f"❌ Dataset not found at {dataset_path}")
        return

    # Analyze dataset
    colorizer.analyze_dataset(dataset_path)

# ========================================
# Main execution for Colab
# ========================================

if __name__ == "__main__":
    print(" Photo Colorization System - Google Colab Version")
    print("Optimized for /kaggle/input/landscape-image-colorization/landscape Images")
    print()
    print("Available functions:")
    print("- colab_analyze_dataset() - Analyze the dataset")
    print("- colab_train() - Train the model")
    print("- colab_test() - Test the model")
    print("- colab_colorize_single() - Colorize a single image")
    print()
    print("Example usage:")
    print("1. colab_analyze_dataset()")
    print("2. colab_train()")
    print("3. colab_test()")
    print("4. colab_colorize_single()")

 Photo Colorization System - Google Colab Version
Optimized for /kaggle/input/landscape-image-colorization/landscape Images

Available functions:
- colab_analyze_dataset() - Analyze the dataset
- colab_train() - Train the model
- colab_test() - Test the model
- colab_colorize_single() - Colorize a single image

Example usage:
1. colab_analyze_dataset()
2. colab_train()
3. colab_test()
4. colab_colorize_single()


In [ ]:
# Copy the code from october/colorizer_colab.py to Colab
# Then run these commands:

# 1. Analyze dataset
colab_analyze_dataset()

📊 Analyzing dataset...
🎨 Colorizer initialized on cuda
📊 Analyzing dataset...
📁 Analyzing landscape dataset structure...
 Found 7129 gray-color pairs
✅ Dataset Analysis Complete:
   📊 Dataset type: Landscape (gray/color pairs)
   📁 Total pairs: 7129
   📋 Formats: .jpg(10)
   Analysis saved to: dataset_analysis_20250822_021103.json


In [ ]:
# 2. Train the model
colab_train()

🚀 Starting Colab training...
🎨 Colorizer initialized on cuda
📁 Using dataset: /kaggle/input/landscape-image-colorization/landscape Images
🚀 Starting training with 20 epochs...
📊 Analyzing dataset...
📁 Analyzing landscape dataset structure...
 Found 7129 gray-color pairs
✅ Dataset Analysis Complete:
   📊 Dataset type: Landscape (gray/color pairs)
   📁 Total pairs: 7129
   📋 Formats: .jpg(10)
   Analysis saved to: dataset_analysis_20250822_021137.json
📁 Found landscape dataset structure: gray/ and color/ folders
📊 Found 7129 valid gray-color pairs
📊 Dataset loaded: 7129 images, 446 batches


Epoch 1/20: 100%|██████████| 446/446 [06:16<00:00,  1.18it/s, Loss=0.0185]


✅ Epoch 1/20 completed in 377.0s. Average Loss: 0.0182


Epoch 2/20: 100%|██████████| 446/446 [05:12<00:00,  1.43it/s, Loss=0.0156]


✅ Epoch 2/20 completed in 312.9s. Average Loss: 0.0146


...

✅ Epoch 18/20 completed in 307.8s. Average Loss: 0.0069


Epoch 19/20: 100%|██████████| 446/446 [05:07<00:00,  1.45it/s, Loss=0.0056]


✅ Epoch 19/20 completed in 307.5s. Average Loss: 0.0068


Epoch 20/20: 100%|██████████| 446/446 [05:08<00:00,  1.45it/s, Loss=0.0069]

✅ Epoch 20/20 completed in 308.1s. Average Loss: 0.0068
💾 Final model saved to model.pth
📊 Training history saved to model_history.json

🎉 Training Complete!
   ⏱️ Total time: 6284.1 seconds
   📊 Epochs completed: 20
   🏆 Best loss: 0.0068
   📈 Final loss: 0.0068
 Training completed!


In [ ]:
# 3. Test the model
colab_test()

🧪 Starting Colab testing...
🎨 Colorizer initialized on cuda
✅ Model loaded from model.pth
 Running model test...
📁 Found 7129 images to process


Processing images:   0%|          | 3/7129 [00:00<04:48, 24.71it/s]

 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/6234.jpg
💾 Saved to: test_results/colorized_6234.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/1269.jpg
💾 Saved to: test_results/colorized_1269.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3863.jpg
💾 Saved to: test_results/colorized_3863.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/6241.jpg
💾 Saved to: test_results/colorized_6241.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/623.jpg
💾 Saved to: test_results/colorized_623.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/2193.jpg


Processing images:   0%|          | 12/7129 [00:00<02:59, 39.72it/s]

💾 Saved to: test_results/colorized_2193.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3750.jpg
💾 Saved to: test_results/colorized_3750.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/2008.jpg
💾 Saved to: test_results/colorized_2008.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/5982.jpg
💾 Saved to: test_results/colorized_5982.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/2081.jpg
💾 Saved to: test_results/colorized_2081.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/6588.jpg
💾 Saved to: test_results/colorized_6588.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3919.jpg
💾 Saved to: test_results/colorized_3919.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/6197.jpg
💾 Saved to: test_results/colorized_6197.jpg
 Processing: /kaggle/input/landscape-im

...

💾 Saved to: test_results/colorized_4490.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3267.jpg
💾 Saved to: test_results/colorized_3267.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/1201.jpg
💾 Saved to: test_results/colorized_1201.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3120.jpg
💾 Saved to: test_results/colorized_3120.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/2867.jpg
💾 Saved to: test_results/colorized_2867.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/5272.jpg
💾 Saved to: test_results/colorized_5272.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/1801.jpg
💾 Saved to: test_results/colorized_1801.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3000.jpg
💾 Saved to: test_results/colorized_3000.jpg
 Processing: /kaggle/input/landscape-im

Processing images: 100%|█████████▉| 7119/7129 [02:14<00:00, 53.52it/s]

💾 Saved to: test_results/colorized_4421.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/5894.jpg
💾 Saved to: test_results/colorized_5894.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/1979.jpg
💾 Saved to: test_results/colorized_1979.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3059.jpg
💾 Saved to: test_results/colorized_3059.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3410.jpg
💾 Saved to: test_results/colorized_3410.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/2085.jpg
💾 Saved to: test_results/colorized_2085.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/4968.jpg
💾 Saved to: test_results/colorized_4968.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/5922.jpg
💾 Saved to: test_results/colorized_5922.jpg
 Processing: /kaggle/input/landscape-im

Processing images: 100%|██████████| 7129/7129 [02:14<00:00, 53.11it/s]

💾 Saved to: test_results/colorized_5254.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/921.jpg
💾 Saved to: test_results/colorized_921.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/4463.jpg
💾 Saved to: test_results/colorized_4463.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/3950.jpg
💾 Saved to: test_results/colorized_3950.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/5303.jpg
💾 Saved to: test_results/colorized_5303.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/728.jpg
💾 Saved to: test_results/colorized_728.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/1656.jpg
💾 Saved to: test_results/colorized_1656.jpg
 Processing: /kaggle/input/landscape-image-colorization/landscape Images/gray/6119.jpg
💾 Saved to: test_results/colorized_6119.jpg
 Processing: /kaggle/input/landscape-image-